In [ ]:
# Take an Ad-Juster campaign delivery report file in .xlsx format and generates a copy of the report with an additionl "SF#" column.
# Each row's SF# is generated by searching for the SF# in the "Order Name" column. 

import pandas as pd
import sys
import re
from openpyxl import load_workbook
from openpyxl.styles import Border, Font, Side, Alignment

# Prompt the user to either enter the file name of an Ad-Juster in .xlsx format located in the same folder as the program file or quit the program.

filename = input('Please enter the file name of an Ad-juster file in .xlsx format or enter "Q" to quit this program: \n\n')

if str.lower(filename) == 'q':
    print('\n Program terminated.')
    sys.exit()

else: pass
    
while True:
    if not filename.endswith('.xlsx'):
        filename = input('Please enter the file name of a .xlsx file only:\n\n')
        continue
    else: break

# Search through all of the worksheets in the .xlsx file, find the first Ad-Juster campaign delivery report, and open that worksheet.

while True:
    try:
        wb = load_workbook(filename)
        for sheet in wb.sheetnames:
            current_sheet = wb.active
            if current_sheet.cell(row = 1, column = 1).value == 'Ad-Juster Campaign Delivery Report':
                name = sheet
                break
            else:
                name = filename[:-5]
        filehandle = pd.ExcelFile(filename)
        df = filehandle.parse(sheetname = name, skiprows = 8, index_col = None)
        print('\n\nUtilizing data from worksheet', name, 'in', filename)
        break
        
    except:
        filename = input('Please enter a valid file name or press "Q" to quit:\n\n')
        if filename.lower() == 'q':
            print('\n Program terminated.')
            sys.exit()
        else:
            continue
            
# Find the column index associated with the "Campaign Identifier" column.

SF_column = df.columns.get_loc('Campaign Identifier')

df.insert(SF_column + 1, 'SF#', '')

# Retrieve the SF# from each row of the "Order Name" column.

def SF_search(order_name):

    SF_num = 0
    
    if re.search('(SF[1-9][\d][\d][\d][\d][\d])', str(order_name)):
        results = re.findall('(SF[1-9][\d][\d][\d][\d][\d])', str(order_name))
        SF_num = str(results[0])[2:]
        int_SF = int(SF_num)
    
    elif re.search('(SF[1-9][\d][\d][\d][\d])', str(order_name)):
        results = re.findall('(SF[1-9][\d][\d][\d][\d])', str(order_name))
        SF_num = str(results[0])[2:]
        int_SF = int(SF_num)
    
    elif re.search('(?<!\d)[1-9][\d][\d][\d][\d](?!\d)', str(order_name)):
        results = re.findall('(?<!\d)[1-9][\d][\d][\d][\d](?!\d)',str(order_name))
        SF_num = results[0]
        int_SF = int(SF_num)
    
    elif re.search('(?<!\d)[1-9][\d][\d][\d][\d][\d](?!\d)', str(order_name)):
        results = re.findall('(?<!\d)[1-9][\d][\d][\d][\d][\d](?!\d)', str(order_name))
        SF_num = results[0]
        int_SF = int(SF_num)
        
    else: int_SF = ''
        
    return(int_SF)

# Open the "AdJuster Template.xlsx" file and copy the header information from the Ad-Juster report.

adjuster = load_workbook(filename)
raw_data = adjuster.active
headers = list()
values = list()

for r in range(1, 8):
    headers.append(raw_data.cell(row = r, column = 1).value)

for r in range(1, 8):
    values.append(raw_data.cell(row = r, column = 2).value)

save_name = 'AdJuster - '+filename[:-5]+'.xlsx'

# Search for the SF# in each row's "Order Name" column and if it is found, write it to the "SF#" column.

df['SF#'] = df.apply(lambda row: SF_search(row['Order Name']), axis = 1)

# Copy the Ad-Juster report with the added "SF#" column into the "Data" worksheet in "AdJuster Template.xlsx" and save it as a new .xlsx file.

writer = pd.ExcelWriter(save_name, engine = 'openpyxl')
df.to_excel(writer, sheet_name = 'Data', index = False, startrow = 8)

writer.save()

report = load_workbook(save_name)

# Write the Ad-Juster report header information to the "Data" worksheet.
# Ensure that the dates in the header are shown in the correct date format.
# Remove any bold type or borders from the column headers and align column header text to the left.

data = report.active

for r in range(1, 8):
    data.cell(row = r, column = 1).value = headers[r - 1]
    
for r in range(1, 8):
    data.cell(row = r, column = 2).value = values[r - 1]

for r in range(4, 7):
    data.cell(row = r, column = 2).number_format = 'mm-dd-yy'
    
for col in range(1, len(df.columns) + 1):
    data.cell(row = 9, column = col).font = Font(bold = False)
    data.cell(row = 9, column = col).border = Border(left=Side(border_style=None))
    data.cell(row = 9, column = col).alignment = Alignment(horizontal='left')

report.save(save_name)

while True:
    print('\n\nThe Ad-Juster report has been saved as', save_name, '\n\n')
    quit = input('Enter any key to quit.')
    break
